In [107]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [108]:
genre = ["Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

In [109]:
data = pd.read_csv('u.item', sep="|", header=None , encoding='latin-1')

In [110]:
data.shape

(1682, 24)

In [111]:
data.columns = ["index", "title", "releaseDate", "", "imdb"] + genre +[""]

In [112]:
data['title']

0                                Toy Story (1995)
1                                GoldenEye (1995)
2                               Four Rooms (1995)
3                               Get Shorty (1995)
4                                  Copycat (1995)
                          ...                    
1677                            Mat' i syn (1997)
1678                             B. Monkey (1998)
1679                         Sliding Doors (1998)
1680                          You So Crazy (1994)
1681    Scream of Stone (Schrei aus Stein) (1991)
Name: title, Length: 1682, dtype: object

In [113]:
data.drop('', axis='columns', inplace=True)

In [114]:
data.iloc[:,4:]

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1678,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1679,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1680,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [115]:
arr = []
for i in range(data.shape[0]):
  arr.append([])
data['Genre'] = arr
data.iloc[:,-1]

0       []
1       []
2       []
3       []
4       []
        ..
1677    []
1678    []
1679    []
1680    []
1681    []
Name: Genre, Length: 1682, dtype: object

In [116]:
for i in range(data.shape[0]):
  for j in range(4, data.shape[1]-1):
      if(data.iloc[i,j] == 1):
        data.iloc[i,-1].append(genre[j-4])
data.iloc[:,-1]

0       [Children's, Comedy, Crime]
1       [Adventure, Animation, War]
2                             [War]
3       [Adventure, Crime, Fantasy]
4       [Documentary, Fantasy, War]
                   ...             
1677                      [Fantasy]
1678                  [Sci-Fi, War]
1679              [Fantasy, Sci-Fi]
1680                        [Crime]
1681                      [Fantasy]
Name: Genre, Length: 1682, dtype: object

In [117]:
for i in range(4, data.shape[1]-1):
  data.drop(genre[i-4], axis='columns', inplace=True)

In [118]:
data

,index,title,releaseDate,imdb,Genre
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,"[Children's, Comedy, Crime]"
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,"[Adventure, Animation, War]"
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,[War]
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,"[Adventure, Crime, Fantasy]"
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),"[Documentary, Fantasy, War]"
...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,[Fantasy]
1678,1679,B. Monkey (1998),06-Feb-1998,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,"[Sci-Fi, War]"
1679,1680,Sliding Doors (1998),01-Jan-1998,http://us.imdb.com/Title?Sliding+Doors+(1998),"[Fantasy, Sci-Fi]"
1680,1681,You So Crazy (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?You%20So%20Cr...,[Crime]


In [119]:
data.describe()

,index
count,1682.000000
mean,841.500000
std,485.695893
min,1.000000
25%,421.250000
50%,841.500000
75%,1261.750000
max,1682.000000


In [120]:
for i in range(data.shape[0]):
  data.iloc[i,1] = data.iloc[i,1][:-7]

In [121]:
data

,index,title,releaseDate,imdb,Genre
0,1,Toy Story,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,"[Children's, Comedy, Crime]"
1,2,GoldenEye,01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,"[Adventure, Animation, War]"
2,3,Four Rooms,01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,[War]
3,4,Get Shorty,01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,"[Adventure, Crime, Fantasy]"
4,5,Copycat,01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),"[Documentary, Fantasy, War]"
...,...,...,...,...,...
1677,1678,Mat' i syn,06-Feb-1998,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,[Fantasy]
1678,1679,B. Monkey,06-Feb-1998,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,"[Sci-Fi, War]"
1679,1680,Sliding Doors,01-Jan-1998,http://us.imdb.com/Title?Sliding+Doors+(1998),"[Fantasy, Sci-Fi]"
1680,1681,You So Crazy,01-Jan-1994,http://us.imdb.com/M/title-exact?You%20So%20Cr...,[Crime]


In [122]:
for i in range(data.shape[0]):
  data.iloc[i,0] -= 1

In [123]:
def combine_features(row):
  str = ""
  str += row['title']+' '
  for val in row['Genre']:
    str += val+' '
  return str[:-1]

In [124]:
data['combined_features'] = data.apply(combine_features, axis = 1)

In [125]:
data

,index,title,releaseDate,imdb,Genre,combined_features
0,0,Toy Story,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,"[Children's, Comedy, Crime]",Toy Story Children's Comedy Crime
1,1,GoldenEye,01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,"[Adventure, Animation, War]",GoldenEye Adventure Animation War
2,2,Four Rooms,01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,[War],Four Rooms War
3,3,Get Shorty,01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,"[Adventure, Crime, Fantasy]",Get Shorty Adventure Crime Fantasy
4,4,Copycat,01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),"[Documentary, Fantasy, War]",Copycat Documentary Fantasy War
...,...,...,...,...,...,...
1677,1677,Mat' i syn,06-Feb-1998,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,[Fantasy],Mat' i syn Fantasy
1678,1678,B. Monkey,06-Feb-1998,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,"[Sci-Fi, War]",B. Monkey Sci-Fi War
1679,1679,Sliding Doors,01-Jan-1998,http://us.imdb.com/Title?Sliding+Doors+(1998),"[Fantasy, Sci-Fi]",Sliding Doors Fantasy Sci-Fi
1680,1680,You So Crazy,01-Jan-1994,http://us.imdb.com/M/title-exact?You%20So%20Cr...,[Crime],You So Crazy Crime


In [126]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(data['combined_features'])

In [127]:
cosine_sim = cosine_similarity(count_matrix)

In [128]:
cosine_sim.shape

(1682, 1682)

In [129]:
def index_from_title(title):
  return data[data.title == title]['index'].values[0]
def title_from_index(index):
  return data[data.index == index]['title'].values[0]

In [137]:
# index_from_title("Get Shorty (1995)")
movie = "Toy Story"
movie_index = index_from_title(movie)
similar_movies = list(enumerate(cosine_sim[movie_index]))

In [138]:
sorted_similar_movies = sorted(similar_movies, key = lambda x:x[1], reverse=True)[1:]

In [139]:
print("Top 10 recommended movies : ")
for i in range(10):
  print(title_from_index(sorted_similar_movies[i][0]))

Top 10 recommended movies : 
Aladdin
Hercules
Jungle2Jungle
Heavyweights
Pinocchio
Matilda
Balto
Goofy Movie, A
Space Jam
Babe
